# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np # numerical computing library
import pandas as pd # pandas datascience library
import requests # library for making HTTP requests in python
import xlsxwriter # library to convert data in jupyter notebooks into excel sheets
import math # math module for basic math computations
from scipy import stats # module for statistical computations

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
len(stocks)

505

In [4]:
# store premium api_key from alpha vantage into a secrets.py file
## from secrets import api_key
# alternatively, store it as an environment variable
import os
from dotenv import load_dotenv

In [5]:
# load the .env file
load_dotenv()

# access the environment variables
api_key = os.getenv('api_key')

# print it to verify
print(f"API key: {api_key}")

API key: 41G44NMW29K8L9Q7


In [6]:
# add .env file to .gitignore folder for extra layer of security
# note that .env file is automatically hidden
# do not commit sensitive information like API keys, database credentials, and other config details to your Git repo

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get `one-year price returns`, `six-month price returns`, `three-month price returns`, `one-month price returns` for each stock in the universe. Here's how.

There are 3 ways to go about getting the price returns:
1. Utilize the `alpha_vantage` library and the `TechIndicator` module, `ROCR` function to derive price returns
2. Utilize the `alpha_vantage` library and the `TimeSeries` module, to compute the price returns based on yearly and monthly time series data - closing price
3. Default API call, `Analytics Fixed Window` function
4. Default API call, `Analytics Sliding Window` function

In [7]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.alphavantage import AlphaVantage

#### (1) Derive Price Returns using Rate of Change (ROC)

* ((price/ previous price) - 1) * 100
* ROC (monthly) / 100 = 1month return

In [8]:
data, meta_data = TechIndicators(key=api_key, 
                                 output_format='pandas').get_roc(symbol='AAPL', 
                                                                interval='monthly', 
                                                                time_period= 12, # arbitrary number, can adjust to get the optimal time_period
                                                                series_type='close')

# time_period: number of data points used to calculate each moving average value
# the higher the time_period
# the smaller the sample output (rows)

In [9]:
meta_data

{'1: Symbol': 'AAPL',
 '2: Indicator': 'Rate of change : ((price/prevPrice)-1)*100',
 '3: Last Refreshed': '2024-06-28',
 '4: Interval': 'monthly',
 '5: Time Period': 12,
 '6: Series Type': 'close',
 '7: Time Zone': 'US/Eastern Time'}

In [10]:
# 12 month moving averages
data

,ROC
date,
2000-12-29,-71.0534
2001-01-31,-58.3229
2001-02-28,-68.1556
2001-03-30,-67.4987
2001-04-30,-58.9070
...,...
2024-02-29,23.2682
2024-03-28,4.5424
2024-04-30,0.9161


In [11]:
# plotting the 12-month Moving Average ROC values

In [12]:
# descriptive statistics 
print(data.describe())

              ROC
count  283.000000
mean    37.793042
std     54.083604
min    -71.053400
25%      1.038400
50%     35.635700
75%     58.152750
max    275.083600


Disadvantages of using the TechnicalIndicator Module of AlphaVantage Library: <br>
* only able to compute 1month return <br>
* able to compute 3month, 6month, 1year returns if you can create custom function - conversion formula required

Q. Why do the values increase across the board as `time_period` increases?

#### (2) Derive Price Returns using Monthly and Yearly TimeSeries Data
* retrieve `daily` and `monthly` closing prices for the past 60 months (5 years) - in case you would like to create rolling values
* build custom functions to convert prices into `1month`, `3month`, `6month`, `1year` returns

In [13]:
daily, daily_meta = TimeSeries(key = api_key, 
                               output_format = 'pandas').get_daily(symbol='AAPl')

In [14]:
daily_meta

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'AAPl',
 '3. Last Refreshed': '2024-06-28',
 '4. Output Size': 'Compact',
 '5. Time Zone': 'US/Eastern'}

In [15]:
daily

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-06-28,215.770,216.0700,210.3000,210.62,82542718.0
2024-06-27,214.690,215.7395,212.3500,214.10,49772707.0
2024-06-26,211.500,214.8600,210.6400,213.25,66213186.0
2024-06-25,209.150,211.3800,208.6100,209.07,56713868.0
2024-06-24,207.720,212.7000,206.5900,208.14,80727006.0
...,...,...,...,...,...
2024-02-12,188.415,188.6700,186.7900,187.15,41781934.0
2024-02-09,188.650,189.9900,188.0000,188.85,45155216.0
2024-02-08,189.385,189.5350,187.3500,188.32,40962046.0


In [16]:
monthly, monthly_meta = TimeSeries(key = api_key,
                                   output_format = 'pandas').get_monthly(symbol='AAPL')

In [17]:
monthly_meta

{'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'AAPL',
 '3. Last Refreshed': '2024-06-28',
 '4. Time Zone': 'US/Eastern'}

In [18]:
monthly

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-06-28,192.900,220.20,192.150,210.62,1.723984e+09
2024-05-31,169.580,193.00,169.110,192.25,1.336570e+09
2024-04-30,171.190,178.36,164.075,170.33,1.240411e+09
2024-03-28,179.550,180.53,168.490,171.48,1.430780e+09
2024-02-29,183.985,191.05,179.250,180.75,1.161712e+09
...,...,...,...,...,...
2000-04-28,135.500,139.50,104.870,124.06,7.734290e+07
2000-03-31,118.560,150.38,114.000,135.81,7.766390e+07
2000-02-29,104.000,119.94,97.000,114.62,6.535520e+07


In [19]:
# custom function that returns 1-month returns
# use the closing price instead of the adjusted closing price to compute returns 
# more concerned about the actual market sentiment instead of the company valuation / performance
# time_period = number of months

def xday_return(time_period): 
    return(daily['4. close'][0] - daily['4. close'][time_period]) / daily['4. close'][0]

def xmonth_return(time_period):
    return (monthly['4. close'][0] - monthly['4. close'][time_period]) / monthly['4. close'][0]

In [20]:
# derive 1month return using closing price before adjustments
# assume there are 21 trading days in a month
xday_return(21)

0.0965245465767734

In [21]:
# derive 3month return using closing price before adjustments
xmonth_return(3)

0.1858323046244422

In [22]:
# derive 6month return using closing price before adjustments
xmonth_return(6)

0.08588927927072454

In [23]:
# derive 12month return using closing price before adjustments
xmonth_return(12)

0.07905232171683603

Q. Why does percentage change increase as the range / time_period increase?

> Limitations of using the StockTimeSeries Module of the AlphaVantage Library: <br> 
* Creation of conversion formula - different options available for consideration
* Selection of optimal parameter values

#### (3) Alpha Vantage API `Advanced Analytics - Fixed Window` Function

In [24]:
# target statistics to retrieve via api call
# '1-year price return', '6-month price return', '3-month price return', '1-month price return' based on daily closing prices
# derivative statistics to compute later on 
# '1-year return percentile', '6-month return percentile', '3-month return percentile', '1-month return percentile', 'HQM score', 'number of shares to buy'

# 1-year return using daily return data, monthly fixed window
# required parameters
function = 'ANALYTICS_FIXED_WINDOW'
symbol_str = 'AAPL'
time_period = '1year'
interval = 'daily' 
calculations = 'cumulative_return'

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

# find out why WEEKLY and MONTHLY intervals do not work despite returning data.status_code = 200

200
{'meta_data': {'symbols': 'AAPL', 'min_dt': '2023-06-30', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'AAPL': 0.09157612133024995}}}}


In [25]:
# 6-month return using daily return data, monthly fixed window
# required parameters
function = 'ANALYTICS_FIXED_WINDOW'
symbol_str = 'AAPL'
time_period = '6month'
interval = 'daily' 
calculations = 'cumulative_return'

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'min_dt': '2024-01-02', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'AAPL': 0.13755486755577118}}}}


In [26]:
# 3-month return using daily return data, monthly fixed window
# required parameters
function = 'ANALYTICS_FIXED_WINDOW'
symbol_str = 'AAPL'
time_period = '3month'
interval = 'daily' 
calculations = 'cumulative_return'

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'min_dt': '2024-04-01', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'AAPL': 0.24041436009259876}}}}


In [27]:
# 1-month return using daily return data, daily fixed window
# required parameters
function = 'ANALYTICS_FIXED_WINDOW'
symbol_str = 'A'
time_period = '1month'
interval = 'daily' 
calculations = 'cumulative_return'

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'A', 'min_dt': '2024-05-30', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'A': -0.013620453507837471}}}}


In [28]:
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


#### (4) Alpha Vantage API `Advanced Analytics - Sliding Window` Function

In [29]:
# target statistics to retrieve via api call
# 'latest_price', '1-year price return', '6-month price return', '3-month price return', '1-month price return'
# derivative statistics to compute later on 
# '1-year return percentile', '6-month return percentile', '3-month return percentile', '1-month return percentile', 'HQM score', 'number of shares to buy'

# 1-year return using daily return data, 250 moving window size
# required parameters
function = 'ANALYTICS_SLIDING_WINDOW'
symbol_str = 'AAPL'
time_period = '1year'
interval = 'daily' 
calculations = 'cumulative_return'
window_size = 250 # assuming there are 21 trading days a month

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&WINDOW_SIZE={window_size}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'window_size': 250, 'min_dt': '2023-06-30', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'RUNNING_CUMULATIVE_RETURN': {'AAPL': {'2024-06-28': 0.09157612133024995}}, 'window_start': {'2024-06-28': '2023-06-30'}}}}}


In [30]:
# 6-month return using daily return data, 123 moving window size
# required parameters
function = 'ANALYTICS_SLIDING_WINDOW'
symbol_str = 'AAPL'
time_period = '6month'
interval = 'daily' 
calculations = 'cumulative_return'
window_size = 123 

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&WINDOW_SIZE={window_size}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'window_size': 123, 'min_dt': '2024-01-02', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'RUNNING_CUMULATIVE_RETURN': {'AAPL': {'2024-06-28': 0.13755486755577118}}, 'window_start': {'2024-06-28': '2024-01-02'}}}}}


In [31]:
# 3-month return using daily return data, 062 moving window size
# required parameters
function = 'ANALYTICS_SLIDING_WINDOW'
symbol_str = 'AAPL'
time_period = '3month'
interval = 'daily' 
calculations = 'cumulative_return'
window_size = 62

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&WINDOW_SIZE={window_size}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'window_size': 62, 'min_dt': '2024-04-01', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'RUNNING_CUMULATIVE_RETURN': {'AAPL': {'2024-06-28': 0.24041436009259876}}, 'window_start': {'2024-06-28': '2024-04-01'}}}}}


In [32]:
# 1-month return using daily return data, 019 moving window size
# required parameters
function = 'ANALYTICS_SLIDING_WINDOW'
symbol_str = 'AAPL'
time_period = '1month'
interval = 'daily' 
calculations = 'cumulative_return'
window_size = 19

api_url = f'https://alphavantage.co/query?function={function}&SYMBOLS={symbol_str}&RANGE={time_period}&INTERVAL={interval}&CALCULATIONS={calculations}&WINDOW_SIZE={window_size}&apikey={api_key}'
r = requests.get(api_url)
data = r.json()
print(r.status_code)
print(data)

200
{'meta_data': {'symbols': 'AAPL', 'window_size': 19, 'min_dt': '2024-05-30', 'max_dt': '2024-06-28', 'ohlc': 'Close', 'interval': 'DAILY'}, 'payload': {'RETURNS_CALCULATIONS': {'CUMULATIVE_RETURN': {'RUNNING_CUMULATIVE_RETURN': {'AAPL': {'2024-06-27': 0.11924303413665083, '2024-06-28': 0.09555266579974031}}, 'window_start': {'2024-06-27': '2024-05-30', '2024-06-28': '2024-05-31'}}}}}


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [33]:
data['payload']['RETURNS_CALCULATIONS']['CUMULATIVE_RETURN']['RUNNING_CUMULATIVE_RETURN']['AAPL']['2024-06-28']

0.09555266579974031

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [34]:
# function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 50))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
# print(symbol_strings[0])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Six-Month Price Return', 'Three-Month Price Return', 'One-Month Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [35]:
# create blank dataframe
df = pd.DataFrame(columns = my_columns)
df

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Number of Shares to Buy


In [36]:
# check if a symbol is valid by attempting a simple API call
def is_symbol_valid(symbol):
    try:
        # fetch the daily data for the symbol
        daily, daily_meta = TimeSeries(key = api_key, 
                                       output_format = 'pandas').get_daily(symbol=symbol)
        return True  # if no error occurs, the symbol is valid
    
    except ValueError as e:
        print(f"Invalid symbol {symbol}: {e}")
        return False  # if an error occurs, the symbol is invalid

In [37]:
# check to see if output is what you intend it to be
is_symbol_valid('A')

True

In [38]:
# check to see if output is what you intend it to be
is_symbol_valid('ANTM')

Invalid symbol ANTM: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.


False

In [39]:
# function to get cumulative return for a given time period
def get_cumulative_return(symbol_str, time_period):
    base_url = 'https://alphavantage.co/query'
    function = 'ANALYTICS_FIXED_WINDOW'
    interval = 'daily'
    calculations = 'cumulative_return'
    
    params = {
        'function': function,
        'SYMBOLS': symbol_str,
        'RANGE': time_period,
        'INTERVAL': interval,
        'CALCULATIONS': calculations,
        'apikey': api_key
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # debug output: print the entire API response
    # print(f"API response for {symbol_str} with time period {time_period}")
    ## print(data)
    
    # initialize a dictionary to store results
    cumulative_returns_dict = {}
    
    # check if the response has the necessary data
    if 'payload' in data and 'RETURNS_CALCULATIONS' in data['payload'] and 'CUMULATIVE_RETURN' in data['payload']['RETURNS_CALCULATIONS']:
        cumulative_returns = data['payload']['RETURNS_CALCULATIONS']['CUMULATIVE_RETURN']
        
        ## print(cumulative_returns)
        
        for symbol in symbol_str.split(','):
            if symbol in cumulative_returns:
                # store the cumulative return for the symbol
                cumulative_returns_dict[symbol] = cumulative_returns[symbol]
                # print(f"{symbol}: {cumulative_returns[symbol]}")
            else:
                # assign np.nan if the symbol doesn't have data
                cumulative_returns_dict[symbol] = np.nan
                ## print(f"No data for {symbol} with time period {time_period}, assigned np.nan")
    else:
        # handle case where the entire batch query failed or individual symbols failed
        error_message = data.get('error', 'Unknown error')
        # print an error message indicating the problem with the data retrieval
        ## print(f"No data for the batch {symbol_str} with time period {time_period}. Error: {error_message}")
        
    
        # check if it's a KeyError for a specific symbol and handle accordingly
        if isinstance(error_message, str) and "No data can be retrieved for ticker" in error_message:
            # parse the error message to get the specific symbol
            error_symbol = error_message.split('ticker ')[1].split(' ')[0]
            ## print(f"Skipping {error_symbol} due to lack of data.")
            
            # assign np.nan for the problematic symbol
            for symbol in symbol_str.split(','):
                if symbol == error_symbol:
                    cumulative_returns_dict[symbol] = np.nan
                    ## print(f"{symbol}: No data available, assigned np.nan")
                else:
                    # retry individually for other symbols
                    individual_result = get_cumulative_return(symbol, time_period)
                    # ensure the individual result is a dictionary and extract the value
                    if isinstance(individual_result, dict):
                        cumulative_returns_dict[symbol] = individual_result.get(symbol, np.nan)
                    else:
                        # if not a dict, assign np.nan
                        cumulative_returns_dict[symbol] = np.nan
        else:
            # if a generic error, assign np.nan to all symbols in the batch
            for symbol in symbol_str.split(','):
                cumulative_returns_dict[symbol] = np.nan
    
    return cumulative_returns_dict.get(symbol)
    

In [40]:
# verify if output is what you intend it to be
get_cumulative_return('A', '1year')

0.08377918348856239

In [41]:
# check to see if output is what you intend it to be
get_cumulative_return('ANTM', '1year')

nan

In [42]:
type(symbol_strings[0])

str

In [43]:
# split the string by commas to get a list of symbols
symbols_list = symbol_strings[0].split(',')

In [44]:
# print the list to verify
print(symbols_list)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB']


In [45]:
for symbol_str in symbol_strings:
    
    # default parameters
    api_key = api_key
    function = 'ANALYTICS_FIXED_WINDOW'
    interval = 'daily' 
    calculations = 'cumulative_return'
    
    # api call for ticker & price data for each synbol
    for symbol in symbol_str.split(','):
        print(f"Symbol: {symbol}, Validity: {is_symbol_valid(symbol)}")
        
        # check if the symbol is valid
        if is_symbol_valid(symbol):
            try:
                # fetch the daily data
                daily, daily_meta = TimeSeries(key=api_key, 
                                               output_format='pandas').get_daily(symbol=symbol)

                # construct the row for valid symbol
                new_row = pd.Series([symbol, 
                                     daily ['4. close'].iloc[0],
                                     get_cumulative_return(symbol, '1year'),
                                     get_cumulative_return(symbol, '6month'),
                                     get_cumulative_return(symbol, '3month'),
                                     get_cumulative_return(symbol, '1month'),
                                     'N/A'], 
                                     index=my_columns)
                
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")

        else:
            print(f"Invalid symbol: {symbol}")

            # construct the row for an invalid symbol (or when an error occurs)
            new_row = pd.Series([symbol, 
                                 np.nan,  # no price data available due to error
                                 get_cumulative_return(symbol, '1year'),
                                 get_cumulative_return(symbol, '6month'),
                                 get_cumulative_return(symbol, '3month'),
                                 get_cumulative_return(symbol, '1month'),
                                 'N/A'], 
                                 index=my_columns)  
            
        # append the new row to df using pd.concat
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
        
df
# you want the value, without the key for get_cumulative_return
# append the rows that return valid values too

Symbol: A, Validity: True
Symbol: AAL, Validity: True
Symbol: AAP, Validity: True
Symbol: AAPL, Validity: True
Symbol: ABBV, Validity: True
Invalid symbol ABC: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.
Symbol: ABC, Validity: False
Invalid symbol ABC: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.
Invalid symbol: ABC
Symbol: ABMD, Validity: True
Symbol: ABT, Validity: True
Symbol: ACN, Validity: True
Symbol: ADBE, Validity: True
Symbol: ADI, Validity: True
Symbol: ADM, Validity: True
Symbol: ADP, Validity: True
Symbol: ADSK, Validity: True
Symbol: AEE, Validity: True
Symbol: AEP, Validity: True
Symbol: AES, Validity: True
Symbol: AFL, Validity: True
Symbol: AIG, Validity: True
Symbol: AIV, Validity: True
Symbol: AIZ, Validity: True
Symbol: AJG, Validity: True
Symbol: AKAM, Validity: True
Symbol: ALB, Validity: True
Symbol: ALG

,Ticker,Price,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Number of Shares to Buy
0,A,129.63,0.083779,-0.064215,-0.109439,-0.01362,N/A
1,AAL,11.33,-0.36845,-0.156994,-0.264763,-0.009615,N/A
2,AAP,63.33,-0.084731,0.029821,-0.256199,-0.054211,N/A
3,AAPL,210.62,0.091576,0.137555,0.240414,0.101051,N/A
4,ABBV,171.52,0.324957,0.093799,-0.042054,0.097307,N/A
...,...,...,...,...,...,...,...
500,YUM,132.46,-0.025383,0.03674,-0.052433,-0.017651,N/A
501,ZBH,108.53,-0.248515,-0.102333,-0.161688,-0.049537,N/A
502,ZBRA,308.93,0.044282,0.15281,0.026346,-0.024288,N/A
503,ZION,43.37,0.685467,0.001545,0.018247,0.026266,N/A


Making 505 x 5 API calls - computationally intensive and inefficient.

In [46]:
# display all rows in the DataFrame
pd.set_option('display.max_rows', None)  # Remove the row limit
print(df)

    Ticker    Price One-Year Price Return Six-Month Price Return  \
0        A   129.63              0.083779              -0.064215   
1      AAL    11.33              -0.36845              -0.156994   
2      AAP    63.33             -0.084731               0.029821   
3     AAPL   210.62              0.091576               0.137555   
4     ABBV   171.52              0.324957               0.093799   
5      ABC      NaN             -0.062273                    NaN   
6     ABMD   381.02                   NaN                    NaN   
7      ABT   103.91               -0.0274              -0.044699   
8      ACN   303.41             -0.001604              -0.118737   
9     ADBE   555.54              0.136097              -0.042288   
10     ADI   228.26               0.19309                0.18945   
11     ADM    60.45             -0.176254              -0.154351   
12     ADP   238.69              0.110729               0.034373   
13    ADSK   247.45              0.209374       

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [47]:
# make a copy of the original DataFrame
oneyear_df = df.copy()

# drop the irrelevant columns from the copy
oneyear_df.drop(columns=['Six-Month Price Return', 'Three-Month Price Return', 'One-Month Price Return'], inplace=True)

# sort the copied DataFrame by 'One-Year Price Return' in descending order
oneyear_df.sort_values('One-Year Price Return', ascending=False, inplace=True)

# retain on the top 50 performing stocks
oneyear_df = oneyear_df[:51]
oneyear_df.reset_index(drop = True, 
                       inplace = True)

oneyear_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,123.54,1.921408,N/A
1,GPS,23.89,1.798082,N/A
2,ANET,350.48,1.162656,N/A
3,NRG,77.86,1.150091,N/A
4,MU,131.53,1.096384,N/A
5,SLG,56.64,1.026737,N/A
6,WDC,75.77,0.997627,N/A
7,LLY,905.38,0.944698,N/A
8,AVGO,1605.53,0.883613,N/A
9,GE,158.97,0.820321,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [48]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 1000000


1000000


In [49]:
position_size = float(portfolio_size) / len(oneyear_df.index)
for i in range(0, len(oneyear_df['Ticker'])):
    oneyear_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / oneyear_df['Price'][i])
oneyear_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,123.54,1.921408,158
1,GPS,23.89,1.798082,820
2,ANET,350.48,1.162656,55
3,NRG,77.86,1.150091,251
4,MU,131.53,1.096384,149
5,SLG,56.64,1.026737,346
6,WDC,75.77,0.997627,258
7,LLY,905.38,0.944698,21
8,AVGO,1605.53,0.883613,12
9,GE,158.97,0.820321,123


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [50]:
# initialise the new dataframe

hqm_columns =  ['Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score']
                
HQM_df = pd.DataFrame(columns = hqm_columns)

In [51]:
HQM_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [52]:
for symbol_str in symbol_strings:
    
    # default parameters
    api_key = api_key
    function = 'ANALYTICS_FIXED_WINDOW'
    interval = 'daily' 
    calculations = 'cumulative_return'
    
    # api call for ticker & price data for each synbol
    for symbol in symbol_str.split(','):
        print(f"Symbol: {symbol}, Validity: {is_symbol_valid(symbol)}")
        
        # check if the symbol is valid
        if is_symbol_valid(symbol):
            try:
                # fetch the daily data
                daily, daily_meta = TimeSeries(key=api_key, 
                                               output_format='pandas').get_daily(symbol=symbol)

                # construct the row for valid symbol
                new_row = pd.Series([symbol, 
                                     daily ['4. close'].iloc[0],
                                     'N/A',
                                     get_cumulative_return(symbol, '1year'),
                                     'N/A',
                                     get_cumulative_return(symbol, '6month'),
                                     'N/A',
                                     get_cumulative_return(symbol, '3month'),
                                     'N/A',
                                     get_cumulative_return(symbol, '1month'),
                                     'N/A',
                                     'N/A'], 
                                     index=hqm_columns)
                
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")

        else:
            print(f"Invalid symbol: {symbol}")

            # construct the row for an invalid symbol (or when an error occurs)
            new_row = pd.Series([symbol, 
                                 'N/A',  # no price data available due to error
                                 'N/A',
                                 get_cumulative_return(symbol, '1year'),
                                 'N/A',
                                 get_cumulative_return(symbol, '6month'),
                                 'N/A',
                                 get_cumulative_return(symbol, '3month'),
                                 'N/A',
                                 get_cumulative_return(symbol, '1month'),
                                 'N/A',
                                 'N/A'], 
                                 index=hqm_columns)  
            
        # append the new row to df using pd.concat
        HQM_df = pd.concat([HQM_df, new_row.to_frame().T], ignore_index=True)
        
HQM_df

Symbol: A, Validity: True
Symbol: AAL, Validity: True
Symbol: AAP, Validity: True
Symbol: AAPL, Validity: True
Symbol: ABBV, Validity: True
Invalid symbol ABC: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.
Symbol: ABC, Validity: False
Invalid symbol ABC: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.
Invalid symbol: ABC
Symbol: ABMD, Validity: True
Symbol: ABT, Validity: True
Symbol: ACN, Validity: True
Symbol: ADBE, Validity: True
Symbol: ADI, Validity: True
Symbol: ADM, Validity: True
Symbol: ADP, Validity: True
Symbol: ADSK, Validity: True
Symbol: AEE, Validity: True
Symbol: AEP, Validity: True
Symbol: AES, Validity: True
Symbol: AFL, Validity: True
Symbol: AIG, Validity: True
Symbol: AIV, Validity: True
Symbol: AIZ, Validity: True
Symbol: AJG, Validity: True
Symbol: AKAM, Validity: True
Symbol: ALB, Validity: True
Symbol: ALG

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.63,N/A,0.083779,N/A,-0.064215,N/A,-0.109439,N/A,-0.01362,N/A,N/A
1,AAL,11.33,N/A,-0.36845,N/A,-0.156994,N/A,-0.264763,N/A,-0.009615,N/A,N/A
2,AAP,63.33,N/A,-0.084731,N/A,0.029821,N/A,-0.256199,N/A,-0.054211,N/A,N/A
3,AAPL,210.62,N/A,0.091576,N/A,0.137555,N/A,0.240414,N/A,0.101051,N/A,N/A
4,ABBV,171.52,N/A,0.324957,N/A,0.093799,N/A,-0.042054,N/A,0.097307,N/A,N/A
5,ABC,N/A,N/A,-0.062273,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
6,ABMD,381.02,N/A,NaN,N/A,NaN,N/A,NaN,N/A,NaN,N/A,N/A
7,ABT,103.91,N/A,-0.0274,N/A,-0.044699,N/A,-0.068304,N/A,0.021329,N/A,N/A
8,ACN,303.41,N/A,-0.001604,N/A,-0.118737,N/A,-0.101877,N/A,0.065344,N/A,N/A
9,ADBE,555.54,N/A,0.136097,N/A,-0.042288,N/A,0.106455,N/A,0.245969,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [53]:
time_periods = ['One-Year', 
                'Six-Month', 
                'Three-Month', 
                'One-Month']

In [54]:
HQM_df.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [55]:
for row in HQM_df.index:
    for time_period in time_periods:
        HQM_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore( HQM_df[f'{time_period} Price Return'].dropna(), 
                                                                                       HQM_df.loc[row, f'{time_period} Price Return']) / 100

In [56]:
# print each percentile score to make sure it was calculated properly
for time_period in time_periods: 
    print(HQM_df[f'{time_period} Return Percentile'])

0      0.453961
1      0.019272
2      0.203426
3      0.479657
4      0.800857
5      0.231263
6           NaN
7      0.269807
8      0.325482
9      0.556745
10      0.64454
11     0.104925
12     0.518201
13     0.674518
14     0.194861
15     0.464668
16       0.1606
17     0.777302
18      0.79015
19      0.27409
20     0.813704
21     0.642398
22     0.336188
23     0.004283
24     0.025696
25     0.057816
26     0.916488
27     0.331906
28          NaN
29     0.957173
30     0.374732
31     0.862955
32     0.379015
33     0.892934
34     0.775161
35      0.38758
36     0.914347
37     0.995717
38     0.278373
39          NaN
40     0.126338
41     0.573876
42      0.17773
43     0.158458
44     0.946467
45     0.027837
46     0.441113
47     0.372591
48     0.550321
49     0.552463
50     0.982869
51     0.766595
52     0.218415
53     0.817987
54     0.635974
55     0.130621
56      0.87152
57     0.053533
58     0.445396
59     0.192719
60     0.156317
61          NaN
62      

In [57]:
HQM_df 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.63,N/A,0.083779,0.453961,-0.064215,0.24026,-0.109439,0.191304,-0.01362,0.306522,N/A
1,AAL,11.33,N/A,-0.36845,0.019272,-0.156994,0.097403,-0.264763,0.019565,-0.009615,0.343478,N/A
2,AAP,63.33,N/A,-0.084731,0.203426,0.029821,0.467532,-0.256199,0.023913,-0.054211,0.084783,N/A
3,AAPL,210.62,N/A,0.091576,0.479657,0.137555,0.722944,0.240414,0.997826,0.101051,0.928261,N/A
4,ABBV,171.52,N/A,0.324957,0.800857,0.093799,0.625541,-0.042054,0.441304,0.097307,0.919565,N/A
5,ABC,N/A,N/A,-0.062273,0.231263,NaN,NaN,NaN,NaN,NaN,NaN,N/A
6,ABMD,381.02,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A
7,ABT,103.91,N/A,-0.0274,0.269807,-0.044699,0.287879,-0.068304,0.334783,0.021329,0.602174,N/A
8,ACN,303.41,N/A,-0.001604,0.325482,-0.118737,0.15368,-0.101877,0.228261,0.065344,0.847826,N/A
9,ADBE,555.54,N/A,0.136097,0.556745,-0.042288,0.296537,0.106455,0.878261,0.245969,0.997826,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [58]:
from statistics import mean

In [59]:
for row in HQM_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(HQM_df.loc[row, f'{time_period} Return Percentile'])
    HQM_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [60]:
HQM_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,129.63,N/A,0.083779,0.453961,-0.064215,0.24026,-0.109439,0.191304,-0.01362,0.306522,0.298012
1,AAL,11.33,N/A,-0.36845,0.019272,-0.156994,0.097403,-0.264763,0.019565,-0.009615,0.343478,0.11993
2,AAP,63.33,N/A,-0.084731,0.203426,0.029821,0.467532,-0.256199,0.023913,-0.054211,0.084783,0.194914
3,AAPL,210.62,N/A,0.091576,0.479657,0.137555,0.722944,0.240414,0.997826,0.101051,0.928261,0.782172
4,ABBV,171.52,N/A,0.324957,0.800857,0.093799,0.625541,-0.042054,0.441304,0.097307,0.919565,0.696817
5,ABC,N/A,N/A,-0.062273,0.231263,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ABMD,381.02,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ABT,103.91,N/A,-0.0274,0.269807,-0.044699,0.287879,-0.068304,0.334783,0.021329,0.602174,0.373661
8,ACN,303.41,N/A,-0.001604,0.325482,-0.118737,0.15368,-0.101877,0.228261,0.065344,0.847826,0.388812
9,ADBE,555.54,N/A,0.136097,0.556745,-0.042288,0.296537,0.106455,0.878261,0.245969,0.997826,0.682342


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [61]:
HQM_df.sort_values(by = 'HQM Score', ascending=False, inplace=True)

# retain on the top 50 performing stocks
HQM_df = HQM_df[:51]
HQM_df.reset_index(drop = True, 
                   inplace = True)

HQM_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,123.54,N/A,1.921408,1.0,1.565194,1.0,0.367313,1.0,0.118102,0.956522,0.98913
1,AVGO,1605.53,N/A,0.883613,0.982869,0.490217,0.982684,0.192973,0.984783,0.180887,0.982609,0.983236
2,ANET,350.48,N/A,1.162656,0.995717,0.51356,0.987013,0.169241,0.967391,0.154186,0.973913,0.981009
3,LLY,905.38,N/A,0.944698,0.985011,0.533996,0.989177,0.192394,0.982609,0.110814,0.95,0.976699
4,NTAP,128.8,N/A,0.726352,0.970021,0.509876,0.984848,0.23205,0.995652,0.105579,0.93913,0.972413
5,IRM,89.62,N/A,0.633794,0.952891,0.326414,0.950216,0.135026,0.936957,0.1336,0.96087,0.950233
6,AMAT,235.99,N/A,0.645498,0.957173,0.534108,0.991342,0.13289,0.932609,0.089822,0.915217,0.949085
7,KLAC,824.51,N/A,0.716184,0.96788,0.477514,0.978355,0.162148,0.96087,0.070611,0.869565,0.944167
8,STX,103.27,N/A,0.727031,0.972163,0.267469,0.915584,0.132866,0.930435,0.109534,0.945652,0.940959
9,LRCX,1064.85,N/A,0.672615,0.961456,0.42673,0.969697,0.086591,0.843478,0.122125,0.958696,0.933332


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [62]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [63]:
# create an explicit copy of the DataFrame
HQM_df = HQM_df.copy()

# convert price to numeric
HQM_df['Price'] = pd.to_numeric(HQM_df['Price'], errors='coerce')

In [64]:
position_size = float(portfolio_size) / len(HQM_df.index)

# iterate over each row in the DataFrame and calculate 'Number of Shares to Buy'
for i in range(len(HQM_df)):
    # make sure we don't try to operate on NaN values
    if not pd.isna(HQM_df.loc[i, 'Price']):
        HQM_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / HQM_df.loc[i, 'Price'])
    else:
        HQM_df.loc[i, 'Number of Shares to Buy'] = np.nan

In [65]:
HQM_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,123.54,158,1.921408,1.0,1.565194,1.0,0.367313,1.0,0.118102,0.956522,0.98913
1,AVGO,1605.53,12,0.883613,0.982869,0.490217,0.982684,0.192973,0.984783,0.180887,0.982609,0.983236
2,ANET,350.48,55,1.162656,0.995717,0.51356,0.987013,0.169241,0.967391,0.154186,0.973913,0.981009
3,LLY,905.38,21,0.944698,0.985011,0.533996,0.989177,0.192394,0.982609,0.110814,0.95,0.976699
4,NTAP,128.80,152,0.726352,0.970021,0.509876,0.984848,0.23205,0.995652,0.105579,0.93913,0.972413
5,IRM,89.62,218,0.633794,0.952891,0.326414,0.950216,0.135026,0.936957,0.1336,0.96087,0.950233
6,AMAT,235.99,83,0.645498,0.957173,0.534108,0.991342,0.13289,0.932609,0.089822,0.915217,0.949085
7,KLAC,824.51,23,0.716184,0.96788,0.477514,0.978355,0.162148,0.96087,0.070611,0.869565,0.944167
8,STX,103.27,189,0.727031,0.972163,0.267469,0.915584,0.132866,0.930435,0.109534,0.945652,0.940959
9,LRCX,1064.85,18,0.672615,0.961456,0.42673,0.969697,0.086591,0.843478,0.122125,0.958696,0.933332


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [66]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
HQM_df.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [67]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [68]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [69]:
writer.close()